## Ajuste

Montagem do drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Bibliotecas

Importação de bibliotecas

In [ ]:
import pandas as pd

## Pré-processamento

Filtro por palavras-chave

In [ ]:
# definir as palavras em inglês
key_words_english = ['flood']

# definir as palavras em alemão
key_words_german = ['hochwasser',
                    'flut',
                    'überschwemmung']

# definir as palavras adicionais em alemão
key_addiotional_words_german = ['deich',
                                'sandsack']

# combinar palavras-chave
key_words = key_words_english + key_words_german + key_addiotional_words_german

In [ ]:
def select_tweets_with_key_words (df):
  return df[df.tweet.str.contains('|'.join(key_words), na=False)]

## Aplicação da técnica

Calcular frequência absoluta

In [ ]:
def calcule_absolute_frequency (tweets):
  '''
  define um dicionário para registrar as frequências absolutas
  onde criamos um dicionário global do dataframe e registramos
  a frequência das palavras-chave 
  '''
  absolute_frequency = {}
  
  for word in key_words:
    sum_word = 0
    for tweet in tweets:
      if (tweet.count(word) > 0): 
        sum_word += 1
    absolute_frequency[word] = sum_word
  
  return absolute_frequency

Calcular frequência relativa

In [ ]:
def calcule_relative_frequency (tweet, word, absolute_frequency):
  '''
  a frequência relativa é a quantidade de aparições de palavras-chave dividio
  pela frequência absoluta dessa palavra-chave
  '''
  return tweet.count(word)/absolute_frequency

Calcular entropia

In [ ]:
import math

def calcule_tweets_entropy (df):
  # definimos um dicionári para a entropia dos tweets
  entropies = {}
  # realizamos a construção do dicionário de frequência absoluta das palavras chave no tweet
  absolute_frequency = calcule_absolute_frequency(df.tweet)
  
  index = df.index

  for id in index:
    frequences_array = []
    for word in key_words:
      if (df.loc[id].tweet.count(word) > 0):
        relative_frequency = calcule_relative_frequency(df.loc[id].tweet, word, absolute_frequency[word])
        frequences_array.append(relative_frequency * math.log2(1/relative_frequency))
    entropies[id] = sum(frequences_array)
  return entropies

## Válidação

**Main**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/TCC_Thalles Sobral/2-dados/germany/use-data/adjusted.csv')
df = df.drop(columns='hashtag')

In [ ]:
# filtrar dataframe apenas com tweets com palavras-chave
tweets_with_key_words = select_tweets_with_key_words(df)

In [ ]:
# calcular entropia dos tweets
tweets_with_entropy = calcule_tweets_entropy(tweets_with_key_words)
# tweets_with_entropy = calcule_tweets_entropy(df)

quantos é maior que 0.5? E quais as situações desses dados?

In [ ]:
keys_array = []
values_array = []
for keys in tweets_with_entropy:
  keys_array.append(keys)
  values_array.append(tweets_with_entropy[keys])

In [ ]:
values_entropy = pd.Series(values_array)

In [ ]:
tweets_with_key_words['entropy'] = values_entropy.values
# df['entropy'] = values_entropy.values

In [ ]:
# df.to_csv('tweets-with-entropy.csv')
tweets_with_key_words.to_csv('tweets-with-entropy.csv')
!cp tweets-with-entropy.csv "drive/My Drive/TCC_Thalles Sobral/2-dados/germany/use-data"